# Target
- create agent => OKAY
- get text content response => OKAY
- get tools call response => OKAY
- get structure ouput => OKAY
- get data sources 
  - url
  - document
- get usage data
- can retrieve data from vector db
- add memory to agent
  - short term
  - long term
- add agent guardrail
- use mcp
- use middleware

## Create basic agent

In [9]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_openai import ChatOpenAI

# model = ChatOpenAI(
#     model="gpt-4.1-nano",
#     temperature=0.1,
#     max_tokens=1000,
#     timeout=30
# )
model = init_chat_model("gpt-4.1-nano", temperature=0.1, max_tokens=1000, timeout=15)

agent = create_agent(model)

### Agent with tools

In [22]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"


@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

tools = [search, get_weather]

agent_with_tools = create_agent(
    model="gpt-4.1-nano",
    tools=[search, get_weather],
    middleware=[handle_tool_errors],
    system_prompt="You are a helpful assistant. Be concise and accurate."
)

### agent with structure output

In [18]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str | None = None
    email: str | None = None
    phone: str | None = None

agent = create_agent(
    model,
    tools=[search],
    response_format=ToolStrategy(ContactInfo),
    system_prompt="You are a helpful assistant. Be concise and accurate. Extract contact info ONLY if present. If missing contact info, return empty string."
)

# BUG: There is a big problem here if the contact info is not present
result = agent.invoke({
    "messages": [
      {"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"},
                #  {"role": "user", "content": "what is machine learning?"}
                 ]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [20]:
from langchain.agents.structured_output import ProviderStrategy

class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model,
    response_format=ProviderStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"},
        {"role": "user", "content": "what is machine learning?"}
                 ]
})

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

### Agent memory

In [25]:

from langchain.agents import AgentState


class CustomState(AgentState):
    user_preferences: dict

agent = create_agent(
    model,
    tools=[search, get_weather],
    state_schema=CustomState
)
# The agent can now track additional state beyond messages
result = agent.invoke({"messages": [{"role": "user", "content": "I prefer technical explanations"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})
result

{'messages': [HumanMessage(content='I prefer technical explanations', additional_kwargs={}, response_metadata={}, id='66d622c2-b00e-4410-85c0-c53be1945d30'),
  AIMessage(content="Understood. Please specify the topic or subject you'd like a technical explanation for.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 65, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'id': 'chatcmpl-CmMza8YKw5e6bwZTLJ1l8Z1fo36Lx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b1897-d3ed-7633-8f0b-54d6144b4c9d-0', usage_metadata={'input_tokens': 65, 'output_tokens': 17, 'total_tokens': 82, 'input_token_details': {'audio

### Streaming response (text content, tool calls)

In [26]:
for chunk in agent_with_tools.stream({
    "messages": [{"role": "user", "content": "Search for AI news and summarize the findings. After that tell me the weather in San Francisco and New York."}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Agent: Search for AI news and summarize the findings. After that tell me the weather in San Francisco and New York.
Calling tools: ['search', 'get_weather', 'get_weather']
Agent: Weather in New York: Sunny, 72°F
Agent: Here's a summary of the AI news: (Note: As I cannot fetch specific news articles directly, please specify if you'd like a summary of recent AI developments or headlines you are interested in.)

The weather in San Francisco is sunny with a temperature of 72°F, and the weather in New York is also sunny with a temperature of 72°F.


### Get source content 